# 6-3. Hybrid RAG (Advanced) - 재난대응 시스템

Graph RAG와 Vector RAG를 결합한 하이브리드 검색 시스템입니다.
- **Graph RAG**: Neo4j 그래프 데이터베이스에서 구조화된 정보 검색
- **Vector RAG**: Chroma 벡터 DB에서 문서 검색
- **Gemini API**: Google Gemini를 사용한 자연어 생성

In [106]:
# 필요한 라이브러리 import
import os
from typing import List, Dict, Optional
from dotenv import load_dotenv
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Neo4j
from neo4j import GraphDatabase

# Chroma
import chromadb
from chromadb.config import Settings

# Gemini API
from google import genai

load_dotenv()
print("라이브러리 로드 완료")


라이브러리 로드 완료


## 1. 데이터베이스 연결 설정


In [107]:
# Neo4j 연결 설정
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USER = os.getenv("NEO4J_USER", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "password")

neo4j_driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def get_neo4j_session():
    return neo4j_driver.session()

# Neo4j 연결 확인
try:
    with get_neo4j_session() as session:
        result = session.run("RETURN 1 as test")
        print(f"✓ Neo4j 연결 성공: {NEO4J_URI}")
except Exception as e:
    print(f"✗ Neo4j 연결 실패: {e}")


✓ Neo4j 연결 성공: bolt://localhost:7687


In [108]:
# Chroma 벡터 DB 연결
CHROMA_PERSIST_DIR = "data/chroma"
DOCS_DIR = "docs"

chroma_client = chromadb.PersistentClient(
    path=CHROMA_PERSIST_DIR,
    settings=Settings(anonymized_telemetry=False)
)

# 컬렉션 가져오기 또는 생성
try:
    chroma_collection = chroma_client.get_collection("disaster_docs")
    print(f"✓ Chroma 컬렉션 로드 성공: {chroma_collection.count()} 문서")
except Exception as e:
    print(f"✗ Chroma 컬렉션 로드 실패: {e}")
    print("새 컬렉션 생성 및 문서 적재 중...")
    
    # 새 컬렉션 생성
    chroma_collection = chroma_client.create_collection(
        name="disaster_docs",
        metadata={"description": "재난 행동요령 문서"}
    )
    
    # docs 폴더의 마크다운 파일 읽기 및 적재
    import glob
    
    doc_files = glob.glob(f"{DOCS_DIR}/*.md")
    print(f"\n찾은 문서 파일: {len(doc_files)}개")
    
    documents = []
    ids = []
    metadatas = []
    
    for doc_file in doc_files:
        try:
            with open(doc_file, 'r', encoding='utf-8') as f:
                content = f.read()
            
            filename = os.path.basename(doc_file)
            doc_id = filename.replace('.md', '')
            
            documents.append(content)
            ids.append(doc_id)
            metadatas.append({"source": filename, "type": "disaster_guide"})
            
            print(f"  ✓ {filename} 적재 완료 ({len(content)}자)")
        except Exception as e:
            print(f"  ✗ {doc_file} 적재 실패: {e}")
    
    # Chroma에 문서 추가
    if documents:
        chroma_collection.add(
            documents=documents,
            ids=ids,
            metadatas=metadatas
        )
        print(f"\n✓ 총 {len(documents)}개 문서를 Chroma에 적재했습니다.")
    else:
        print("\n✗ 적재할 문서가 없습니다.")


✗ Chroma 컬렉션 로드 실패: Collection [disaster_docs] does not exist


## 1-1. Neo4j 데이터 적재 (필요시)


In [ ]:
# Neo4j 데이터 적재 함수 (배치 처리로 성능 향상)
def load_neo4j_data(nodes_csv: str, relationships_csv: str, session):
    """CSV 파일에서 Neo4j로 데이터를 적재합니다."""
    
    # 기존 데이터 삭제
    session.run("MATCH (n) DETACH DELETE n")
    print("✓ 기존 데이터 삭제 완료")
    
    # 노드 CSV 읽기
    nodes_df = pd.read_csv(nodes_csv, encoding='utf-8-sig')
    print(f"\n노드 CSV 로드: {len(nodes_df)}개")
    
    # 노드 타입별로 배치 적재
    node_types = nodes_df['type'].unique()
    
    for node_type in node_types:
        type_nodes = nodes_df[nodes_df['type'] == node_type]
        print(f"\n{node_type} 노드 적재 중... ({len(type_nodes)}개)")
        
        # 배치 크기 설정
        batch_size = 100
        
        for batch_start in range(0, len(type_nodes), batch_size):
            batch_end = min(batch_start + batch_size, len(type_nodes))
            batch = type_nodes.iloc[batch_start:batch_end]
            
            # 배치별 쿼리 생성
            queries = []
            for _, row in batch.iterrows():
                props = {}
                for col in type_nodes.columns:
                    if col not in ['id', 'type'] and pd.notna(row[col]):
                        value = row[col]
                        # 숫자 변환 시도
                        try:
                            if isinstance(value, (int, float)) or (isinstance(value, str) and value.replace('.', '').replace('-', '').isdigit()):
                                value = float(value) if '.' in str(value) else int(float(value))
                        except:
                            pass
                        props[col] = value
                
                # CREATE 쿼리 작성
                prop_strs = [f"{k}: ${k}" for k in props.keys()]
                create_props = ", ".join(["id: $id"] + prop_strs)
                query = f"CREATE (n:{node_type} {{{create_props}}})"
                params = {"id": row['id'], **props}
                queries.append((query, params))
            
            # 배치 실행
            for query, params in queries:
                try:
                    session.run(query, **params)
                except Exception as e:
                    print(f"  ⚠ 오류: {params.get('id')} - {e}")
            
            if (batch_start // batch_size + 1) % 10 == 0:
                print(f"  진행 중: {batch_end}/{len(type_nodes)} ({batch_end*100//len(type_nodes)}%)")
        
        print(f"  ✓ {node_type} 노드 적재 완료")
    
    # 관계 CSV 읽기 및 배치 적재
    relationships_df = pd.read_csv(relationships_csv, encoding='utf-8-sig')
    print(f"\n관계 CSV 로드: {len(relationships_df)}개")
    
    batch_size = 500
    for batch_start in range(0, len(relationships_df), batch_size):
        batch_end = min(batch_start + batch_size, len(relationships_df))
        batch = relationships_df.iloc[batch_start:batch_end]
        
        for _, rel in batch.iterrows():
            query = f"""
            MATCH (from:{rel['from_type']} {{id: $from_id}})
            MATCH (to:{rel['to_type']} {{id: $to_id}})
            MERGE (from)-[:{rel['relationship_type']}]->(to)
            """
            try:
                session.run(query, from_id=rel['from_id'], to_id=rel['to_id'])
            except Exception as e:
                print(f"  ⚠ 관계 오류: {rel['from_id']} -> {rel['to_id']} - {e}")
        
        if (batch_start // batch_size + 1) % 5 == 0:
            print(f"  진행 중: {batch_end}/{len(relationships_df)} ({batch_end*100//len(relationships_df)}%)")
    
    print(f"  ✓ 관계 적재 완료: {len(relationships_df)}개")
    
    # 최종 통계
    node_count = session.run("MATCH (n) RETURN count(n) as count").single()["count"]
    rel_count = session.run("MATCH ()-[r]->() RETURN count(r) as count").single()["count"]
    print(f"\n✓ Neo4j 데이터 적재 완료: 노드 {node_count}개, 관계 {rel_count}개")

# 데이터 상태 확인 및 적재
with get_neo4j_session() as session:
    # 현재 상태 확인
    node_count = session.run("MATCH (n) RETURN count(n) as count").single()["count"]
    rel_count = session.run("MATCH ()-[r]->() RETURN count(r) as count").single()["count"]
    
    # 예상 데이터 개수 (CSV 확인)
    NODES_CSV = "data/processed/neo4j_nodes.csv"
    REL_CSV = "data/processed/neo4j_relationships.csv"
    
    expected_nodes = 0
    expected_rels = 0
    
    if os.path.exists(NODES_CSV):
        nodes_df = pd.read_csv(NODES_CSV, encoding='utf-8-sig')
        expected_nodes = len(nodes_df)
    
    if os.path.exists(REL_CSV):
        rels_df = pd.read_csv(REL_CSV, encoding='utf-8-sig')
        expected_rels = len(rels_df)
    
    print(f"\n{'='*60}")
    print("Neo4j 데이터 상태 확인")
    print(f"{'='*60}")
    print(f"현재 노드: {node_count}개 (예상: {expected_nodes}개)")
    print(f"현재 관계: {rel_count}개 (예상: {expected_rels}개)")
    
    # 데이터가 없거나 불완전한 경우 적재
    if node_count == 0 or node_count < expected_nodes * 0.9:  # 90% 미만이면 재적재
        if node_count > 0:
            print(f"\n⚠ 데이터가 불완전합니다. 재적재를 진행합니다...")
        else:
            print(f"\n데이터 적재 시작...")
        
        if os.path.exists(NODES_CSV) and os.path.exists(REL_CSV):
            load_neo4j_data(NODES_CSV, REL_CSV, session)
        else:
            print(f"\n✗ CSV 파일을 찾을 수 없습니다:")
            print(f"  - {NODES_CSV}")
            print(f"  - {REL_CSV}")
            print("  먼저 3_preprocessing.ipynb를 실행하여 데이터를 생성하세요.")
    else:
        print(f"\n✓ Neo4j 데이터가 정상적으로 적재되어 있습니다.")
        
        # 노드 타입별 통계
        print("\n노드 타입별 통계:")
        for label in session.run("CALL db.labels()").values():
            label_name = label[0]
            count = session.run(f"MATCH (n:{label_name}) RETURN count(n) as count").single()["count"]
            print(f"  - {label_name}: {count}개")
        
        # 관계 통계 확인
        print("\n관계 통계:")
        rel_stats = session.run("""
        MATCH ()-[r]->()
        RETURN type(r) as rel_type, count(r) as count
        ORDER BY count DESC
        """).data()
        for stat in rel_stats:
            print(f"  - {stat['rel_type']}: {stat['count']}개")
        
        # 예시: 강남구 관계 확인 (IN 관계)
        print("\n강남구 관계 확인 (예시):")
        gangnam_test = session.run("""
        MATCH (s:Shelter)-[r:IN]->(a:Admin {gu: '강남구'})
        RETURN count(s) as shelter_count
        """).single()
        if gangnam_test:
            print(f"  - 강남구 Shelter 노드 수 (IN 관계): {gangnam_test['shelter_count']}개")
        else:
            print("  - 강남구 관계를 찾을 수 없습니다.")


In [109]:
# Gemini API 클라이언트 설정
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY 환경변수를 설정해주세요.")

gemini_client = genai.Client(api_key=GOOGLE_API_KEY)
print("✓ Gemini API 클라이언트 초기화 완료")


✓ Gemini API 클라이언트 초기화 완료


## 2. Neo4j 스키마 정보 가져오기


In [110]:
def get_neo4j_schema(session) -> str:
    """Neo4j 그래프 스키마 정보를 가져옵니다."""
    
    # 노드 라벨 조회
    node_labels_query = "CALL db.labels()"
    node_labels = [record["label"] for record in session.run(node_labels_query)]
    
    # 관계 타입 조회
    rel_types_query = "CALL db.relationshipTypes()"
    rel_types = [record["relationshipType"] for record in session.run(rel_types_query)]
    
    # 노드별 속성 정보 조회
    node_properties = {}
    for label in node_labels:
        props_query = f"""
        MATCH (n:{label})
        RETURN keys(n) as props
        LIMIT 1
        """
        result = session.run(props_query)
        record = result.single()
        if record:
            node_properties[label] = record["props"]
    
    # 스키마 문자열 생성
    schema_parts = ["# Neo4j Graph Schema\n\n"]
    
    schema_parts.append("## Node Labels:\n")
    for label in node_labels:
        props = node_properties.get(label, [])
        props_str = ", ".join(props) if props else "(no properties)"
        schema_parts.append(f"- {label}: {{{props_str}}}")
    
    schema_parts.append("\n## Relationship Types:\n")
    for rel_type in rel_types:
        schema_parts.append(f"- {rel_type}")
    
    return "\n".join(schema_parts)

# 스키마 정보 확인
with get_neo4j_session() as session:
    schema = get_neo4j_schema(session)
    print(schema)


# Neo4j Graph Schema


## Node Labels:

- Admin: {gu, dong}
- Shelter: {lon, open, lat, id, type, name}
- Hazard: {kind, id, active}
- Alert: {ts, type, id}

## Relationship Types:

- IN
- INTERSECTS
- LOCATED_IN


## 3. Graph RAG 검색 함수


In [ ]:
def generate_cypher_query(question: str, schema: str, client: genai.Client) -> str:
    """Gemini를 사용하여 자연어 질문을 Cypher 쿼리로 변환합니다."""
    
    prompt = f"""
당신은 Neo4j Cypher 쿼리 전문가입니다.
다음 그래프 스키마 정보를 참고하여, 사용자의 자연어 질문을 Cypher 쿼리로 변환하세요.

# Graph Schema:
{schema}

# 사용자 질문:
{question}

지침:
1. 질문의 의도를 정확히 파악하여 적절한 Cypher 쿼리를 생성하세요.
2. 노드 라벨과 관계 타입을 정확히 사용하세요. **중요: 관계 타입은 반드시 IN만 사용하세요. LOCATED_IN은 절대 사용하지 마세요.**
3. Admin 노드(행정구역)와 다른 노드(Shelter, TemporaryHousing 등) 간의 관계는 **IN** 관계 타입을 사용하세요.
4. 예시:
   - "강남구에 있는 대피소" → MATCH (s:Shelter)-[:IN]->(a:Admin {{gu: '강남구'}}) RETURN s
   - "강남구 대피소 개수" → MATCH (s:Shelter)-[:IN]->(a:Admin {{gu: '강남구'}}) RETURN count(s)
   - "서초구 대피소" → MATCH (s:Shelter)-[:IN]->(a:Admin {{gu: '서초구'}}) RETURN s
5. 쿼리 결과는 가독성 있게 반환하도록 RETURN 절을 작성하세요.
6. 쿼리만 반환하고 설명은 제외하세요.

Cypher 쿼리:
"""
    
    try:
        response = client.models.generate_content(
            model='gemini-2.0-flash-exp',
            contents=prompt.strip()
        )
        
        cypher_query = response.text.strip()
        # 코드 블록 제거 (```cypher 등)
        if cypher_query.startswith("```"):
            lines = cypher_query.split("\n")
            cypher_query = "\n".join(lines[1:-1]) if len(lines) > 2 else cypher_query
        
        return cypher_query
    except Exception as e:
        print(f"Cypher 쿼리 생성 오류: {e}")
        return None


In [112]:
def graph_rag_search(question: str, schema: str, client: genai.Client, session) -> Dict:
    """Graph RAG 검색: Neo4j에서 구조화된 정보를 검색합니다."""
    
    # 1. Cypher 쿼리 생성
    cypher_query = generate_cypher_query(question, schema, client)
    
    if not cypher_query:
        return {"query": None, "results": [], "error": "Cypher 쿼리 생성 실패"}
    
    print(f"\n[생성된 Cypher 쿼리]\n{cypher_query}\n")
    
    # 2. 쿼리 실행
    try:
        result = session.run(cypher_query)
        records = [dict(record) for record in result]
        
        return {
            "query": cypher_query,
            "results": records,
            "count": len(records)
        }
    except Exception as e:
        return {
            "query": cypher_query,
            "results": [],
            "error": str(e)
        }


## 4. Vector RAG 검색 함수


In [113]:
def vector_rag_search(question: str, collection, client: genai.Client, top_k: int = 5) -> Dict:
    """Vector RAG 검색: Chroma에서 관련 문서를 검색합니다."""
    
    if collection is None:
        return {"results": [], "error": "Chroma 컬렉션이 없습니다."}
    
    try:
        # 질문을 임베딩으로 변환하여 검색 (Chroma가 자동으로 처리)
        results = collection.query(
            query_texts=[question],
            n_results=top_k
        )
        
        documents = []
        for i in range(len(results["ids"][0])):
            doc_id = results["ids"][0][i]
            doc_text = results["documents"][0][i] if results["documents"] else ""
            distance = results["distances"][0][i] if results["distances"] else None
            
            documents.append({
                "id": doc_id,
                "text": doc_text,
                "distance": distance
            })
        
        return {
            "results": documents,
            "count": len(documents)
        }
    except Exception as e:
        return {
            "results": [],
            "error": str(e)
        }


## 5. Hybrid RAG: Graph + Vector 병합


In [114]:
def format_graph_results(graph_results: Dict) -> str:
    """Graph RAG 결과를 텍스트로 포맷팅합니다."""
    
    if graph_results.get("error"):
        return f"오류: {graph_results['error']}"
    
    if not graph_results.get("results"):
        return "검색 결과가 없습니다."
    
    result_parts = ["# Graph RAG 검색 결과\n"]
    result_parts.append(f"Cypher 쿼리: {graph_results.get('query', 'N/A')}\n")
    result_parts.append(f"결과 개수: {graph_results.get('count', 0)}\n")
    
    for i, record in enumerate(graph_results["results"][:10], 1):  # 최대 10개만 표시
        result_parts.append(f"\n## 결과 {i}")
        for key, value in record.items():
            result_parts.append(f"- {key}: {value}")
    
    return "\n".join(result_parts)


def format_vector_results(vector_results: Dict) -> str:
    """Vector RAG 결과를 텍스트로 포맷팅합니다."""
    
    if vector_results.get("error"):
        return f"오류: {vector_results['error']}"
    
    if not vector_results.get("results"):
        return "검색 결과가 없습니다."
    
    result_parts = ["# Vector RAG 검색 결과\n"]
    result_parts.append(f"결과 개수: {vector_results.get('count', 0)}\n")
    
    for i, doc in enumerate(vector_results["results"], 1):
        result_parts.append(f"\n## 문서 {i} (거리: {doc.get('distance', 'N/A')})")
        result_parts.append(f"{doc.get('text', '')[:500]}...")  # 처음 500자만
    
    return "\n".join(result_parts)


In [115]:
def hybrid_rag(question: str, schema: str, client: genai.Client, neo4j_session, chroma_collection) -> Dict:
    """Hybrid RAG: Graph RAG와 Vector RAG를 결합하여 검색하고 답변을 생성합니다."""
    
    # 1. Graph RAG 검색
    print("=" * 60)
    print("[Graph RAG 검색 중...]")
    graph_results = graph_rag_search(question, schema, client, neo4j_session)
    
    # 2. Vector RAG 검색
    print("\n" + "=" * 60)
    print("[Vector RAG 검색 중...]")
    vector_results = vector_rag_search(question, chroma_collection, client)
    
    # 3. 결과 포맷팅
    graph_text = format_graph_results(graph_results)
    vector_text = format_vector_results(vector_results)
    
    # 4. Gemini를 사용하여 최종 답변 생성
    print("\n" + "=" * 60)
    print("[Gemini를 통한 최종 답변 생성 중...]")
    
    final_prompt = f"""
당신은 재난대응 전문가입니다. 사용자의 질문에 대해 다음 검색 결과를 바탕으로 정확하고 도움이 되는 답변을 생성하세요.

# 사용자 질문:
{question}

# Graph RAG 검색 결과 (구조화된 데이터):
{graph_text}

# Vector RAG 검색 결과 (문서):
{vector_text}

지침:
1. Graph RAG 결과와 Vector RAG 결과를 모두 참고하여 종합적인 답변을 제공하세요.
2. 구체적인 데이터(위치, 개수 등)가 있다면 명확히 언급하세요.
3. 답변 마지막에 참고한 근거(Graph RAG의 Cypher 쿼리 경로 및 Vector RAG의 문서 출처)를 명시하세요.
4. 시민이 이해하기 쉽고 실행 가능한 행동 지침을 제공하세요.

답변:
"""
    
    try:
        response = client.models.generate_content(
            model='gemini-2.0-flash-exp',
            contents=final_prompt.strip()
        )
        
        final_answer = response.text.strip()
        
        return {
            "question": question,
            "graph_results": graph_results,
            "vector_results": vector_results,
            "answer": final_answer
        }
    except Exception as e:
        return {
            "question": question,
            "graph_results": graph_results,
            "vector_results": vector_results,
            "answer": None,
            "error": str(e)
        }


## 6. 테스트 질의 실행


In [116]:
# Neo4j 스키마 가져오기
with get_neo4j_session() as session:
    schema = get_neo4j_schema(session)
    
    # 시민 시나리오 테스트 질의
    test_questions = [
        "강남구에 있는 대피소는 몇 개인가요?",
        "지진이 발생했을 때 가까운 대피소를 찾고 싶어요. 서초구 근처 대피소를 알려주세요.",
        "임시주거시설은 어디에 있나요?",
        "지진 발생 시 어떻게 행동해야 하나요?",
        "공습경보가 발령되면 어떻게 해야 하나요?",
        "서초구의 옥외대피소 중 가장 큰 시설은 어디인가요?"
    ]
    
    # 첫 번째 질의 테스트
    question = test_questions[0]
    print(f"\n{'='*60}")
    print(f"질문: {question}")
    print(f"{'='*60}\n")
    
    result = hybrid_rag(
        question=question,
        schema=schema,
        client=gemini_client,
        neo4j_session=session,
        chroma_collection=chroma_collection
    )
    
    print("\n" + "=" * 60)
    print("[최종 답변]")
    print("=" * 60)
    print(result["answer"])



질문: 강남구에 있는 대피소는 몇 개인가요?

[Graph RAG 검색 중...]
Cypher 쿼리 생성 오류: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 50\nPlease retry in 52.317952592s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'gl

In [117]:
# 여러 질의 일괄 실행
with get_neo4j_session() as session:
    schema = get_neo4j_schema(session)
    
    results = []
    for i, question in enumerate(test_questions, 1):
        print(f"\n\n{'#'*60}")
        print(f"질의 {i}/{len(test_questions)}: {question}")
        print(f"{'#'*60}\n")
        
        result = hybrid_rag(
            question=question,
            schema=schema,
            client=gemini_client,
            neo4j_session=session,
            chroma_collection=chroma_collection
        )
        
        results.append(result)
        
        print(f"\n{'='*60}")
        print("[최종 답변]")
        print(f"{'='*60}")
        print(result.get("answer", "답변 생성 실패"))
        print(f"\n{'='*60}\n")




############################################################
질의 1/6: 강남구에 있는 대피소는 몇 개인가요?
############################################################

[Graph RAG 검색 중...]
Cypher 쿼리 생성 오류: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 50\nPlease retry in 52.071375471s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_c

## 7. 결과 분석


In [118]:
# 결과 요약
print("\n" + "="*60)
print("# Hybrid RAG 검색 결과 요약")
print("="*60 + "\n")

for i, result in enumerate(results, 1):
    question = result.get("question", "N/A")
    graph_count = result.get("graph_results", {}).get("count", 0)
    vector_count = result.get("vector_results", {}).get("count", 0)
    has_answer = result.get("answer") is not None
    
    print(f"질의 {i}: {question}")
    print(f"  - Graph RAG: {graph_count}개 결과")
    print(f"  - Vector RAG: {vector_count}개 결과")
    print(f"  - 최종 답변: {'✓ 생성됨' if has_answer else '✗ 생성 실패'}")
    print()



# Hybrid RAG 검색 결과 요약

질의 1: 강남구에 있는 대피소는 몇 개인가요?
  - Graph RAG: 0개 결과
  - Vector RAG: 0개 결과
  - 최종 답변: ✗ 생성 실패

질의 2: 지진이 발생했을 때 가까운 대피소를 찾고 싶어요. 서초구 근처 대피소를 알려주세요.
  - Graph RAG: 0개 결과
  - Vector RAG: 0개 결과
  - 최종 답변: ✗ 생성 실패

질의 3: 임시주거시설은 어디에 있나요?
  - Graph RAG: 0개 결과
  - Vector RAG: 0개 결과
  - 최종 답변: ✗ 생성 실패

질의 4: 지진 발생 시 어떻게 행동해야 하나요?
  - Graph RAG: 0개 결과
  - Vector RAG: 0개 결과
  - 최종 답변: ✗ 생성 실패

질의 5: 공습경보가 발령되면 어떻게 해야 하나요?
  - Graph RAG: 0개 결과
  - Vector RAG: 0개 결과
  - 최종 답변: ✗ 생성 실패

질의 6: 서초구의 옥외대피소 중 가장 큰 시설은 어디인가요?
  - Graph RAG: 0개 결과
  - Vector RAG: 0개 결과
  - 최종 답변: ✗ 생성 실패



In [119]:
# Neo4j 연결 종료
neo4j_driver.close()
print("✓ Neo4j 연결 종료")


✓ Neo4j 연결 종료
